In [1]:
import os
import pandas as pd 
import numpy as np
import re

In [2]:
train = pd.read_csv('./数据集/科创企业成长潜力模型/label/train_2.csv')
train.rename(columns={'label':'Y'}, inplace = True)
test = pd.read_csv('./数据集/科创企业成长潜力模型/upload.csv')

data_all = pd.concat((train[['ID','Y']],test[['ID','Y']]))

In [3]:
data_all

,ID,Y
0,corp_1001749,0.0
1,corp_1001231,0.0
2,corp_1004522,0.0
3,corp_1001738,1.0
4,corp_1003948,1.0
...,...,...
517,corp_1000552,NaN
518,corp_1001079,NaN
519,corp_1004730,NaN
520,corp_1000350,NaN


In [4]:
folder_path = './数据集/科创企业成长潜力模型/data'  # 替换为你的文件夹路径
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx') and file.startswith('其他') ]

In [5]:
len(excel_files)

6

In [6]:
excel_files

['其他_国家科技成果表.xlsx',
 '其他_备案核准投资项目.xlsx',
 '其他_科技成果转化项目.xlsx',
 '其他_科技成果转化项目基本信息.xlsx',
 '其他_药物临床试验信息.xlsx',
 '其他_重点研发计划立项项目.xlsx']

In [7]:
def get_peo_num(x):
    pattern = r"姓名_\d+"
    matches = re.findall(pattern, x)
    count = len(matches)
    return count

df1 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[0],index_col=0)
df1.rename(columns={'成果完成单位':'ID'}, inplace = True)
df1['成果完成人'] = df1['成果完成人'].apply(get_peo_num)

features = df1.groupby('ID')['成果名称'].count().reset_index(drop=False)
features.rename(columns= {'成果名称':'国家科技成果_成果个数'},inplace=True)
type_class = df1['成果类型'].unique()
type_class = type_class[~pd.isna(type_class)]
for year in range(2015,2022):
    sub_data =df1[df1['成果登记年份']==str(year)].groupby('ID')['成果完成人'].count().reset_index(drop=False)
    sub_data.rename(columns= {'成果完成人':f'{year}年_成果完成人数'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    for cl in type_class:
        sub_data = df1[(df1['成果类型']== cl)&(df1['成果登记年份']==year)].groupby('ID')['成果完成人'].sum().reset_index(drop=False)
        sub_data.rename(columns={'成果完成人':f'{year}年_国家科技成果_{cl}_完成人数量'},inplace=True)
        features = features.merge(sub_data,how='left',on='ID')
        
data_all = data_all.merge(features,how='left',on='ID')

In [8]:
def get_num(x):
    if isinstance(x, str):
        numbers = re.findall(r'\d+(?:\.\d+)?', x)
        if numbers:
            extracted_number = float(numbers[0])
            return extracted_number
        else:
            return None
    else:
        return None
def get_zichou(x):
    if isinstance(x, str):
        if '自筹' in x:
            return 1
        else:
            return 0
    else:
        return None
    
df2 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[1],index_col=0)
df2['审批时间'] = df2['审批时间'].dt.year
df2 = df2[df2['申报单位或法人单位（公示名称）'].notna()]
df2.rename(columns={'申报单位或法人单位（公示名称）':'ID'}, inplace = True)
df2['自筹'] = df2['总投资'].apply(get_zichou)
df2['总投资'] = df2['总投资'].apply(get_num)


features = df2.groupby('ID')['项目名称'].count().reset_index(drop=False)
features.rename(columns= {'项目名称':'备案核准投资项目_项目名称'},inplace=True)
type_class = [0,1]
for year in range(2018,2023):
    sub_data = df2[df2['审批时间']==year].groupby('ID')['项目名称'].count().reset_index(drop=False)
    sub_data.rename(columns= {'项目名称':f'{year}年_备案核准投资项目名称数'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df2[df2['审批时间']==year].groupby('ID')['总投资'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'总投资':f'{year}年_备案核准投资项目总投资'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df2[df2['审批时间']==year].groupby('ID')['自筹'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'自筹':f'{year}年_备案核准投资项目自筹数'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
data_all = data_all.merge(features,how='left',on='ID')

In [9]:
df3 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[2],index_col=0)
df3.rename(columns={'原名单公司现名称':'ID'}, inplace = True)
df3 = df3.merge(data_all,on='ID',how='left')
df3['级别'] = df3.groupby('级别')['Y'].transform('mean').fillna(0)
df3['分类'] = df3.groupby('分类')['Y'].transform('mean').fillna(0)
df3['类型'] = df3.groupby('类型')['Y'].transform('mean').fillna(0)
features = df3.groupby('ID').agg({'级别':'sum','分类':'sum','类型':'sum','项目名称':'count'}).reset_index()
features.rename(columns= {'项目名称':'科技成果转化项目_项目个数','级别':'科技成果转化项目_级别','分类':'科技成果转化项目_分类','类型':'科技成果转化项目_类型'},inplace=True)

for year in range(2018,2023):
    sub_data = df3[df3['年度']==year].groupby('ID')['项目名称'].count().reset_index(drop=False)
    sub_data.rename(columns= {'项目名称':f'{year}年_科技成果转化项目数'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df3[df3['年度']==year].groupby('ID')['级别'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'级别':f'{year}年_科技成果转化项目级别'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df3[df3['年度']==year].groupby('ID')['分类'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'分类':f'{year}年_科技成果转化项目分类'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df3[df3['年度']==year].groupby('ID')['类型'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'类型':f'{year}年_科技成果转化项目类型'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
data_all = data_all.merge(features,how='left',on='ID')

In [10]:
df4 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[3],index_col=0)
df4.rename(columns={'公司名称':'ID'}, inplace = True)
df4 = df4.merge(data_all,on='ID',how='left')

df4['级别'] = df4.groupby('级别')['Y'].transform('mean').fillna(0)
df4['分类'] = df4.groupby('分类')['Y'].transform('mean').fillna(0)
df4['类型'] = df4.groupby('类型')['Y'].transform('mean').fillna(0)
features = df4.groupby('ID').agg({'级别':'sum','分类':'sum','类型':'sum','项目名称':'count'}).reset_index()
features.rename(columns= {'项目名称':'科技成果转化项目基本信息_项目个数','级别':'科技成果转化项目基本信息_级别','分类':'科技成果转化项目基本信息_分类','类型':'科技成果转化项目基本信息_类型'},inplace=True)

for year in range(2018,2023):
    sub_data = df4[df4['发布年度']==year].groupby('ID')['项目名称'].count().reset_index(drop=False)
    sub_data.rename(columns= {'项目名称':f'{year}年_科技成果转化项目数'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df4[df4['发布年度']==year].groupby('ID')['级别'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'级别':f'{year}年_科技成果转化项目级别'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df4[df4['发布年度']==year].groupby('ID')['分类'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'分类':f'{year}年_科技成果转化项目分类'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df4[df4['发布年度']==year].groupby('ID')['类型'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'类型':f'{year}年_科技成果转化项目类型'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
data_all = data_all.merge(features,how='left',on='ID')

In [11]:
def get_num(x):
    if isinstance(x, str):
        numbers = re.findall(r'\d+(?:\.\d+)?', x)
        if numbers:
            extracted_number = float(numbers[0])
            return extracted_number
        else:
            return None
    else:
        return None

In [12]:
df5 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[4],index_col=0)
df5.rename(columns={'企业名称':'ID'}, inplace = True)
df5 = df5[df5['ID'].notna()]
df5['试验公示日期'] = df5['试验公示日期'].apply(lambda x: x[:4])
df5['目标入组人数'] = df5['目标入组人数'].apply(get_num)
df5 = df5.merge(data_all,on='ID',how='left')

df5['剂型'] = df5.groupby('剂型')['Y'].transform('mean').fillna(0)
df5['试验状态'] = df5.groupby('试验状态')['Y'].transform('mean').fillna(0)

features = df5.groupby('ID').agg({'剂型':'sum','试验状态':'sum','目标入组人数':'sum','产品名称':'count'}).reset_index()
for year in range(2018,2023):
    sub_data = df5[df5['试验公示日期']==year].groupby('ID')['产品名称'].count().reset_index(drop=False)
    sub_data.rename(columns= {'产品名称':f'{year}年_药物临床项目数'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df5[df5['试验公示日期']==year].groupby('ID')['剂型'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'剂型':f'{year}年_药物临床剂型'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df5[df5['试验公示日期']==year].groupby('ID')['试验状态'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'试验状态':f'{year}年_药物临床试验状态'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
    
    sub_data = df5[df5['试验公示日期']==year].groupby('ID')['目标入组人数'].sum().reset_index(drop=False)
    sub_data.rename(columns= {'目标入组人数':f'{year}年_药物临床目标入组人数'},inplace=True)
    features = features.merge(sub_data,how='left',on='ID')
data_all = data_all.merge(features,how='left',on='ID')

In [13]:
df6 = pd.read_excel('./数据集/科创企业成长潜力模型/data/'+excel_files[5],index_col=0)
df6.rename(columns={'企业名称':'ID'}, inplace = True)
df6 = df6[df6['ID'].notna()]

df6 = df6.merge(data_all,on='ID',how='left')
df6['项目类型'] = df6.groupby('项目类型')['Y'].transform('mean').fillna(0)

features = df6.groupby('ID').agg({'项目类型':'sum','项目名称':'count'}).reset_index()
data_all = data_all.merge(features,how='left',on='ID')

In [15]:
data_all.to_csv('other_new.csv',index=0)